In [ ]:
# imports

import os
from dotenv import load_dotenv
from huggingface_hub import login
from pricer.evaluator import evaluate
from litellm import completion
from pricer.items import Item
import numpy as np
from tqdm.notebook import tqdm
import csv
from sklearn.feature_extraction.text import HashingVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingLR


c:\Users\vijib\projects\llm_eng_manual\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LITE_MODE = True

load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


In [4]:
y = np.array([float(item.price) for item in train])
documents = [item.summary for item in train]


In [5]:
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = HashingVectorizer(n_features=5000,stop_words='english',binary=True)
X=vectorizer.fit_transform(documents)

In [6]:
X.shape
len(documents)

20000

In [7]:
class NeuralNetwork(nn.Module):
  def __init__(self,input_size):
    super(NeuralNetwork,self).__init__() #initializes the parent neural network so our class objects are treated as Pytorch class Objects
    self.layer1 = nn.Linear(input_size,128)
    self.layer2 = nn.Linear(128, 64)
    self.layer3 = nn.Linear(64, 64)
    self.layer4 = nn.Linear(64, 64)
    self.layer5 = nn.Linear(64, 64)
    self.layer6 = nn.Linear(64, 64)
    self.layer7 = nn.Linear(64, 64)
    self.layer8 = nn.Linear(64, 1)
    self.relu = nn.ReLU()
  
  def forward(self,x):
    output1 = self.relu(self.layer1(x))
    output2 = self.relu(self.layer2(output1))
    output3 = self.relu(self.layer3(output2))
    output4 = self.relu(self.layer4(output3))
    output5 = self.relu(self.layer5(output4))
    output6 = self.relu(self.layer6(output5))
    output7 = self.relu(self.layer7(output6))
    output8 = self.layer8(output7)
    return output8


In [8]:
torch.FloatTensor(y).unsqueeze(1).shape


torch.Size([20000, 1])

In [9]:
# Covert data to Pytorch tensors
X_train_tensor = torch.FloatTensor(X.toarray())
y_train_tensor = torch.FloatTensor(y).unsqueeze(1)

X_train_tensor = torch.FloatTensor(X.toarray())
y_train_tensor = torch.FloatTensor(y).unsqueeze(1)

X_train,X_val,y_train,y_val = train_test_split(X_train_tensor, y_train_tensor, test_size=0.01, random_state=42)

train_dataset = TensorDataset(X_train,y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [10]:
input_size = X_train_tensor.shape[1]
model = NeuralNetwork(input_size)

In [11]:
model.parameters()

<generator object Module.parameters at 0x000001DA2728B760>

In [12]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of trainable parameters: {trainable_params:,}")

Number of trainable parameters: 669,249


In [13]:
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
from tqdm import tqdm

EPOCHS=2

for epoch in range(EPOCHS):
  model.train()
  for batch_X,batch_y in tqdm(train_loader):
    optimizer.zero_grad()

    outputs = model(batch_X)
    loss = loss_function(outputs,batch_y)
    loss.backward() # calculates the gradients
    optimizer.step() # asks each weight to take a step towards reducing the error by gradient
  
  model.eval()
  with torch.no_grad():
    val_outputs = model(X_val)
    val_loss = loss_function(val_outputs,y_val)
  print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {loss.item():.3f}, Val Loss: {val_loss.item():.3f}')

100%|██████████| 310/310 [00:04<00:00, 73.04it/s]


Epoch [1/2], Train Loss: 28364.539, Val Loss: 19755.146


100%|██████████| 310/310 [00:04<00:00, 67.02it/s]

Epoch [2/2], Train Loss: 13321.362, Val Loss: 17863.285


In [14]:
def neural_network(item):
  model.eval()
  with torch.no_grad():
    vector = vectorizer.transform([item.summary])
    vector = torch.FloatTensor(vector.toarray())
    result = model(vector)[0].item()
  return max(0, result)

In [15]:
from pricer.evalsp import evaluate

evaluate(neural_network, test)

 56%|█████▌    | 111/200 [00:00<00:00, 1101.76it/s]

$51 $67 $20 $29 $47 $172 $10 $80 $33 $60 $437 $95 $81 $163 $32 $34 $13 $24 $62 $71 $21 $58 $89 $46 $266 $251 $210 $32 $59 $43 $50 $170 $30 $22 $154 $273 $67 $168 $105 $65 $162 $86 $14 $68 $92 $64 $97 $63 $7 $1 $8 $41 $123 $28 $134 $47 $39 $154 $18 $38 $78 $15 $33 $1 $381 $112 $33 $257 $21 $215 $2 $33 $98 $91 $16 $61 $66 $45 $32 $54 $57 $160 $29 $39 $12 $54 $37 $141 $134 $105 $27 $86 $22 $12 $31 $77 $65 $55 $91 $184 $28 $67 $3 $41 $9 $62 $115 $234 $2 $49 $17 $46 $123 $35 $17 $145 $126 $58 $84 $23 $18 $280 $28 $37 $51 $21 $25 $203 $84 $38 $46 $146 $133 $39 $89 $19 $102 $85 $29 $59 $44 $134 $9 $197 $244 $84 $64 $277 $22 $12 $18 $214 $15 $61 $27 $103 $152 $10 $27 $14 $80 $7 $2 $23 $462 

100%|██████████| 200/200 [00:00<00:00, 1238.36it/s]


$17 $88 $14 $22 $53 $11 $23 $243 $49 $28 $8 $21 $0 $24 $115 $344 $22 $107 $18 $32 $89 $25 $42 $25 $8 $60 $60 $91 $49 $4 $39 $88 $12 $2 $14 

In [16]:
def messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [{"role": "user", "content": message}]

In [17]:
print(test[0].summary)

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.


In [23]:
model = 'groq/openai/gpt-oss-120b'
from litellm import completion

response = completion(model=model,messages=messages_for(test[0]),reasoning_effort='low')
msg = response.choices[0].message.content
print(msg)

$199


In [27]:
response = completion(api_base='http://localhost:11534',messages=messages_for(test[0]),model="ollama/llama3.2:1b")

response.choices[0].message.content


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



APIConnectionError: litellm.APIConnectionError: OllamaException - [WinError 10061] No connection could be made because the target machine actively refused it

In [32]:
ollama_model = "ollama/llama3.2:latest"
response = completion(messages=messages_for(test[0]),model=ollama_model,api_base="http://localhost:11434")

In [33]:
response.choices[0].message.content

'$349-$399'

In [34]:
test[0].price

219.0